In [1]:
import pandas as pd
x_train=pd.read_csv('training_input_qFI2jug.csv', index_col="Horodate_UTC")
x_train.index = pd.DatetimeIndex(x_train.index)

x_train.head()


,IDS,Horodate,Mois,Injection RTE (W),Refoulement RTE (W),Pertes modélisées (W),Consommations totale (W),Consommation HTA télérelevée à courbe de charge (W),Consommation totale profilée (W),Consommation HTA profilée (W),...,Production décentralisée profilée (W),Production photovoltaïque profilée (W),Production autre profilée (W),Température réalisée lissée (°Celcius),Température normale lissée (°Celcius),Production éolienne totale (W),Production photovoltaïque totale (W),Pseudo rayonnement,Consommation HTA totale (W),Soutirage net vers autres GRD (W)
Horodate_UTC,,,,,,,,,,,,,,,,,,,,,
2013-10-13 22:00:00,1,2013-10-13T00:00:00+02:00,10,39784230000,103415000,2849582877,37576289504,8062590100,29513699404,1009515939,...,11876221,0.0,608873.0,9.6,13.5,302936853.0,19660.0,NaN,9072106039,436423300
2013-10-13 22:30:00,2,2013-10-13T00:30:00+02:00,10,37114681000,119543000,2603906116,35230706411,8023402840,27207303571,1007708541,...,11719006,0.0,608873.0,9.6,13.5,368725682.0,17004.0,NaN,9031111381,407924660
2013-10-13 23:00:00,3,2013-10-13T01:00:00+02:00,10,36148675000,136447000,2516689035,34428316838,7982839990,26445476848,966660739,...,11742773,0.0,608873.0,9.6,13.5,448572685.0,24299.0,NaN,8949500729,398239990
2013-10-13 23:30:00,4,2013-10-13T01:30:00+02:00,10,35488831000,164083000,2457628530,33922151279,7953531370,25968619909,929311479,...,11719010,0.0,608873.0,9.5,13.5,550243295.0,23308.0,NaN,8882842849,380571680
2013-10-14 00:00:00,5,2013-10-13T02:00:00+02:00,10,34882911000,191859000,2403762298,33447730318,7922448360,25525281958,907509517,...,11731807,0.0,608873.0,9.5,13.5,644783673.0,21963.0,NaN,8829957877,367153030


I think, We need to use the pseudo radiation column in the time series analysis.
And we can drop this column when we will train the model.


In [2]:
liste_dates = x_train.index.tolist()
for dt in liste_dates :
    x_train.at[dt, 'sort'] = dt.strftime('%m%d%H%S')

x_train['sort'].astype(int)

df_month = x_train.sort_values(by='sort', ascending=True)
df_month.head(20)
df_month = df_month.interpolate(str="linear", axis=0).ffill().bfill()
df = df_month.sort_values(by='IDS', ascending=True)

x_train.drop('sort', axis=1)

x_train['Pseudo rayonnement'] = df['Pseudo rayonnement']


x_train.head(20)

,IDS,Horodate,Mois,Injection RTE (W),Refoulement RTE (W),Pertes modélisées (W),Consommations totale (W),Consommation HTA télérelevée à courbe de charge (W),Consommation totale profilée (W),Consommation HTA profilée (W),...,Production photovoltaïque profilée (W),Production autre profilée (W),Température réalisée lissée (°Celcius),Température normale lissée (°Celcius),Production éolienne totale (W),Production photovoltaïque totale (W),Pseudo rayonnement,Consommation HTA totale (W),Soutirage net vers autres GRD (W),sort
Horodate_UTC,,,,,,,,,,,,,,,,,,,,,
2013-10-13 22:00:00,1,2013-10-13T00:00:00+02:00,10,39784230000,103415000,2849582877,37576289504,8062590100,29513699404,1009515939,...,0.0,608873.0,9.6,13.5,3.029369e+08,19660.0,68.988889,9072106039,436423300,10132200
2013-10-13 22:30:00,2,2013-10-13T00:30:00+02:00,10,37114681000,119543000,2603906116,35230706411,8023402840,27207303571,1007708541,...,0.0,608873.0,9.6,13.5,3.687257e+08,17004.0,68.977778,9031111381,407924660,10132200
2013-10-13 23:00:00,3,2013-10-13T01:00:00+02:00,10,36148675000,136447000,2516689035,34428316838,7982839990,26445476848,966660739,...,0.0,608873.0,9.6,13.5,4.485727e+08,24299.0,68.877778,8949500729,398239990,10132300
2013-10-13 23:30:00,4,2013-10-13T01:30:00+02:00,10,35488831000,164083000,2457628530,33922151279,7953531370,25968619909,929311479,...,0.0,608873.0,9.5,13.5,5.502433e+08,23308.0,68.822222,8882842849,380571680,10132300
2013-10-14 00:00:00,5,2013-10-13T02:00:00+02:00,10,34882911000,191859000,2403762298,33447730318,7922448360,25525281958,907509517,...,0.0,608873.0,9.5,13.5,6.447837e+08,21963.0,68.755556,8829957877,367153030,10140000
2013-10-14 00:30:00,6,2013-10-13T02:30:00+02:00,10,32989918000,218749000,2237743365,31811786665,7890473020,23921313645,911513262,...,0.0,608873.0,9.5,13.4,7.440487e+08,23600.0,68.766667,8801986282,345164660,10140000
2013-10-14 01:00:00,7,2013-10-13T03:00:00+02:00,10,31555221000,286655000,2114206795,30572424807,7878131750,22694293057,897252387,...,0.0,608873.0,9.4,13.4,8.640563e+08,20637.0,68.644444,8775384137,326120710,10140100
2013-10-14 01:30:00,8,2013-10-13T03:30:00+02:00,10,30358247000,332130000,2012649959,29543363726,7869134200,21674229526,899179610,...,0.0,608873.0,9.4,13.4,9.622568e+08,19622.0,68.655556,8768313810,313636680,10140100
2013-10-14 02:00:00,9,2013-10-13T04:00:00+02:00,10,29634878000,373161000,1951941973,28919999723,7857298140,21062701583,903294176,...,0.0,608873.0,9.4,13.4,1.032753e+09,17640.0,68.600000,8760592316,303554290,10140200


In [3]:
from jours import holydays

date = holydays()

date.Dataframe(x_train)

date.split()

In [4]:
workday_df = date.business_days()

workday_df.head(20)


,IDS,Horodate,Mois,Injection RTE (W),Refoulement RTE (W),Pertes modélisées (W),Consommations totale (W),Consommation HTA télérelevée à courbe de charge (W),Consommation totale profilée (W),Consommation HTA profilée (W),...,Production autre profilée (W),Température réalisée lissée (°Celcius),Température normale lissée (°Celcius),Production éolienne totale (W),Production photovoltaïque totale (W),Pseudo rayonnement,Consommation HTA totale (W),Soutirage net vers autres GRD (W),sort,workday
Horodate_UTC,,,,,,,,,,,,,,,,,,,,,
2013-10-13 22:00:00,1,2013-10-13T00:00:00+02:00,10,39784230000,103415000,2849582877,37576289504,8062590100,29513699404,1009515939,...,608873.0,9.6,13.5,3.029369e+08,19660.0,68.988889,9072106039,436423300,10132200,1.0
2013-10-13 22:30:00,2,2013-10-13T00:30:00+02:00,10,37114681000,119543000,2603906116,35230706411,8023402840,27207303571,1007708541,...,608873.0,9.6,13.5,3.687257e+08,17004.0,68.977778,9031111381,407924660,10132200,1.0
2013-10-13 23:00:00,3,2013-10-13T01:00:00+02:00,10,36148675000,136447000,2516689035,34428316838,7982839990,26445476848,966660739,...,608873.0,9.6,13.5,4.485727e+08,24299.0,68.877778,8949500729,398239990,10132300,1.0
2013-10-13 23:30:00,4,2013-10-13T01:30:00+02:00,10,35488831000,164083000,2457628530,33922151279,7953531370,25968619909,929311479,...,608873.0,9.5,13.5,5.502433e+08,23308.0,68.822222,8882842849,380571680,10132300,1.0
2013-10-14 00:00:00,5,2013-10-13T02:00:00+02:00,10,34882911000,191859000,2403762298,33447730318,7922448360,25525281958,907509517,...,608873.0,9.5,13.5,6.447837e+08,21963.0,68.755556,8829957877,367153030,10140000,1.0
2013-10-14 00:30:00,6,2013-10-13T02:30:00+02:00,10,32989918000,218749000,2237743365,31811786665,7890473020,23921313645,911513262,...,608873.0,9.5,13.4,7.440487e+08,23600.0,68.766667,8801986282,345164660,10140000,1.0
2013-10-14 01:00:00,7,2013-10-13T03:00:00+02:00,10,31555221000,286655000,2114206795,30572424807,7878131750,22694293057,897252387,...,608873.0,9.4,13.4,8.640563e+08,20637.0,68.644444,8775384137,326120710,10140100,1.0
2013-10-14 01:30:00,8,2013-10-13T03:30:00+02:00,10,30358247000,332130000,2012649959,29543363726,7869134200,21674229526,899179610,...,608873.0,9.4,13.4,9.622568e+08,19622.0,68.655556,8768313810,313636680,10140100,1.0
2013-10-14 02:00:00,9,2013-10-13T04:00:00+02:00,10,29634878000,373161000,1951941973,28919999723,7857298140,21062701583,903294176,...,608873.0,9.4,13.4,1.032753e+09,17640.0,68.600000,8760592316,303554290,10140200,1.0


In [5]:
dayoff_df = date.public_holiday()

dayoff_df.head(20)


,IDS,Horodate,Mois,Injection RTE (W),Refoulement RTE (W),Pertes modélisées (W),Consommations totale (W),Consommation HTA télérelevée à courbe de charge (W),Consommation totale profilée (W),Consommation HTA profilée (W),...,Production autre profilée (W),Température réalisée lissée (°Celcius),Température normale lissée (°Celcius),Production éolienne totale (W),Production photovoltaïque totale (W),Pseudo rayonnement,Consommation HTA totale (W),Soutirage net vers autres GRD (W),sort,workday
Horodate_UTC,,,,,,,,,,,,,,,,,,,,,
2013-11-11 00:00:00,1349,2013-11-10T01:00:00+01:00,11,35739679000,1881986000,2480031936,36737587529,8103939650,28633647879,1244705830,...,10612909.0,11.2,9.5,3.566297e+09,38332.0,47.059036,9348645480,412192300,11110000,0.0
2013-11-11 00:30:00,1350,2013-11-10T01:30:00+01:00,11,35107502000,1878093000,2423687357,36189321363,8065680450,28123640913,1204531702,...,10612909.0,11.2,9.5,3.572519e+09,37989.0,47.065596,9270212152,389173330,11110000,0.0
2013-11-11 01:00:00,1351,2013-11-10T02:00:00+01:00,11,34601363000,1876619000,2378852503,35618363411,8041900540,27576462871,1171551361,...,10612909.0,11.2,9.5,3.456727e+09,36319.0,47.091834,9213451901,379210310,11110100,0.0
2013-11-11 01:30:00,1352,2013-11-10T02:30:00+01:00,11,32890471000,1906594000,2229116680,33883270780,8003694770,25879576010,1190343728,...,10612909.0,11.2,9.5,3.296739e+09,38013.0,47.085275,9194038498,351906270,11110100,0.0
2013-11-11 02:00:00,1353,2013-11-10T03:00:00+01:00,11,31607584000,1838465000,2118680880,32553570501,7998223120,24555347381,1186092623,...,10612909.0,11.2,9.5,3.056457e+09,34682.0,47.170550,9184315743,332254690,11110200,0.0
2013-11-11 02:30:00,1354,2013-11-10T03:30:00+01:00,11,30484902000,1820289000,2023330963,31383024892,7962023970,23421000922,1202458306,...,10612909.0,11.2,9.5,2.886344e+09,39722.0,47.124632,9164482276,322058310,11110200,0.0
2013-11-11 03:00:00,1355,2013-11-10T04:00:00+01:00,11,29847959000,1895013000,1969772442,30778071498,7978172100,22799899398,1216948383,...,10612909.0,11.2,9.4,2.932993e+09,39383.0,47.223026,9195120483,321517970,11110300,0.0
2013-11-11 03:30:00,1356,2013-11-10T04:30:00+01:00,11,29384503000,2016119000,1931046403,30350629097,7981246200,22369382897,1212461197,...,10612909.0,11.2,9.4,3.061554e+09,37357.0,47.209907,9193707397,326890260,11110300,0.0
2013-11-11 04:00:00,1357,2013-11-10T05:00:00+01:00,11,29489337000,2121915000,1939788229,30535526718,8017763690,22517763028,1198219251,...,10612909.0,11.1,9.4,3.268023e+09,39020.0,47.249265,9215982941,329819630,11110400,0.0
